In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#import oshttp://localhost:8996/notebooks/inceptionv_keras_vanilla.ipynb#
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [10]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
import math
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
#from keras.utils import multi_gpu_model

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2576782667799773140
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 78577664
locality {
  bus_id: 1
}
incarnation: 15464607011721760332
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:05:00.0, compute capability: 5.2"
]


In [12]:
train_data_dir = "train"
val_data_dir = "test"
img_width = 180
img_height = 180
#classnames = pickle.load(open("/path/to/val/", "rb"))
batch_size = 64   # 258
train_size = 1192270
test_size = 132499

In [13]:
# Data generator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True,
        class_mode = 'categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        class_mode = 'categorical')

Found 1192270 images belonging to 27 classes.
Found 132499 images belonging to 27 classes.


In [14]:
# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False, input_tensor=None, input_shape=(img_width, img_height, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

x = Dropout(0.1)(x)

# and a logistic layer -- let's say we have 200 classes

predictions = Dense(27, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')



# Replicates `model` on 4 GPUs.
# This assumes that your machine has 4 available GPUs.
#model = multi_gpu_model(model, gpus=4)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[top_k_categorical_accuracy, 'accuracy'])




83689472/83683744 [==============================] - 7s 0us/step


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00001)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=0, mode='auto')

filepath="weights_xception.best.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbd = TensorBoard(log_dir='./logs_xception',  batch_size=batch_size, write_graph=True )

In [ ]:
# train the model on the new data for a few epochs
history = model.fit_generator(generator=train_generator,
                         steps_per_epoch = 7000,
                         epochs = 10,
                         use_multiprocessing=False,
                         max_queue_size=10,
                         workers = 8,     
                         validation_data = validation_generator,
                         callbacks=[tbd, checkpoint, earlystop ],
                         validation_steps = math.ceil(test_size  / batch_size))
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

Epoch 1/10
6999/7000 [============================>.] - ETA: 0s - loss: 1.7118 - top_k_categorical_accuracy: 0.8056 - acc: 0.5120

In [ ]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('xception fully connected layer accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#SVG(model_to_dot(model).create(prog='dot', format='svg'))

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:106]:
   layer.trainable = False
for layer in model.layers[106:]:
   layer.trainable = True


In [ ]:
model.save_weights('model_wpart_xception.h5') 

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001, momentum=0.9), 
              metrics=[top_k_categorical_accuracy, 'accuracy'])
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history1 = model.fit_generator(generator=train_generator,
                         steps_per_epoch = 7000,
                         epochs = 25,
                         initial_epoch=10,
                         use_multiprocessing=False,
                         max_queue_size=10,
                         workers = 16,                                  
                         validation_data = validation_generator,
                         callbacks=[tbd, checkpoint, earlystop, reduce_lr ],
                         validation_steps = math.ceil(test_size / batch_size))

In [ ]:
print(history1.history.keys())
plt.plot(history1.history['acc'])
plt.plot(history1.history['val_acc'])
plt.title('xception model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save_weights('model_xception_wfull.h5') 